# -*- coding: utf-8 -*-
"""youtube_transcriber_v17.ipynb

A script to download YouTube videos and transcribe them using OpenAI's Whisper API.
Designed to run in Google Colab.
"""


In [1]:
import subprocess
import sys

def install_requirements():
    """Install required packages using pip."""
    try:
        # Install system packages
        subprocess.check_call(['apt-get', 'update'])
        subprocess.check_call(['apt-get', 'install', '-y', 'ffmpeg'])

        # Install Python packages
        packages = [
            'yt-dlp',
            'openai',
            'pydub',
            'google-colab'
        ]
        for package in packages:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", package])
    except Exception as e:
        print(f"Error installing requirements: {str(e)}")
        raise


In [2]:


# Install required packages
install_requirements()

import yt_dlp
import openai
import os
import shutil
from pathlib import Path
import sys
import logging
from datetime import datetime
import re
from pydub import AudioSegment
import math
from google.colab import drive
from google.colab import userdata


In [3]:

# VARS: Set default variables
DEFAULT_VERSION = 'v36'
DEFAULT_BASE_DIR = "/content/drive/My Drive/python-projects/kaggle_experiments/transcriber/"
default_youtube_url = "https://www.youtube.com/watch?v=cdiD-9MMpb0"


In [4]:

# Mount Google Drive if not already mounted
if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


In [6]:
# Set up OpenAI client
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=OPENAI_API_KEY)


In [7]:
def setup_directory(base_dir, version_num, clean_dir=True):
    """
    Set up the working directory.

    Args:
        base_dir (str): Base directory path
        version_num (str): Version number for directory naming
        clean_dir (bool): Whether to delete and recreate directory if it exists

    Returns:
        str: Full path of working directory
    """
    dir_nm = "transcriber" + "_" + version_num
    full_dir = os.path.join(base_dir, dir_nm)

    if os.path.exists(full_dir) and clean_dir:
        logger.info(f"Removing existing directory: {full_dir}")
        shutil.rmtree(full_dir)

    os.makedirs(full_dir, exist_ok=True)
    logger.info(f"Using directory: {full_dir}")
    return full_dir


In [8]:

class AudioProcessingError(Exception):
    """Custom exception for audio processing errors."""
    pass


In [9]:
def split_audio(audio_path, chunk_duration_minutes=10):
    """
    Split audio file into smaller chunks.

    Args:
        audio_path (Path): Path to the audio file
        chunk_duration_minutes (int): Duration of each chunk in minutes

    Returns:
        list: List of paths to chunk files
    """
    logger.info(f"Splitting audio file: {audio_path}")
    chunk_paths = []

    try:
        # Load audio file
        audio = AudioSegment.from_mp3(str(audio_path))

        # Calculate duration and number of chunks
        duration_ms = len(audio)
        chunk_duration_ms = chunk_duration_minutes * 60 * 1000
        num_chunks = math.ceil(duration_ms / chunk_duration_ms)

        logger.info(f"Audio duration: {duration_ms/1000/60:.2f} minutes")
        logger.info(f"Splitting into {num_chunks} chunks of {chunk_duration_minutes} minutes each")

        # Create chunks directory
        chunks_dir = audio_path.parent / "chunks"
        chunks_dir.mkdir(exist_ok=True)

        # Split audio into chunks
        for i in range(num_chunks):
            start_ms = i * chunk_duration_ms
            end_ms = min((i + 1) * chunk_duration_ms, duration_ms)

            chunk = audio[start_ms:end_ms]
            chunk_path = chunks_dir / f"chunk_{i+1}.mp3"
            chunk.export(str(chunk_path), format="mp3")

            chunk_paths.append(chunk_path)
            logger.info(f"Created chunk {i+1}/{num_chunks}: {chunk_path}")

    except Exception as e:
        logger.error(f"Error splitting audio: {str(e)}")
        raise AudioProcessingError(f"Failed to split audio: {str(e)}")

    return chunk_paths


In [10]:
def validate_youtube_url(url):
    """
    Validate if the URL is a valid YouTube URL.

    Args:
        url (str): URL to validate

    Returns:
        bool: True if valid, False otherwise
    """
    youtube_regex = (
        r'(https?://)?(www\.)?'
        '(youtube|youtu|youtube-nocookie)\.(com|be)/'
        '(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})')

    match = re.match(youtube_regex, url)
    return bool(match)


In [11]:
def download_youtube_audio(url, output_path="audio"):
    """
    Download audio from a YouTube video.

    Args:
        url (str): YouTube video URL
        output_path (str): Directory to save the audio file

    Returns:
        str: Path to the downloaded audio file
    """
    if not validate_youtube_url(url):
        raise ValueError("Invalid YouTube URL provided")

    # Create output directory if it doesn't exist
    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)

    # Generate timestamp-based filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_template = str(output_path / f"audio_{timestamp}.%(ext)s")

    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': output_template,
        'quiet': False,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            audio_file = str(output_path / f"audio_{timestamp}.mp3")

            if not os.path.exists(audio_file):
                raise FileNotFoundError(f"Audio file not found at {audio_file}")

            return audio_file

    except Exception as e:
        logger.error(f"Error downloading audio: {str(e)}")
        raise AudioProcessingError(f"Failed to download audio: {str(e)}")


In [12]:
def transcribe_audio_chunk(chunk_path, client):
    """
    Transcribe an audio chunk using OpenAI's Whisper API.

    Args:
        chunk_path (Path): Path to the audio chunk
        client: OpenAI client instance

    Returns:
        str: Transcribed text
    """
    if not chunk_path.exists():
        raise FileNotFoundError(f"Audio chunk not found: {chunk_path}")

    try:
        with chunk_path.open('rb') as audio:
            response = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio,
                response_format="text"
            )
            return response
    except Exception as e:
        logger.error(f"Error transcribing chunk {chunk_path}: {str(e)}")
        raise


In [13]:
def save_transcript(transcript):
    """
    Save the transcript to a text file.

    Args:
        transcript (str): Transcribed text

    Returns:
        str: Path to the saved transcript file
    """
    output_file = os.path.join(full_path_nm, "transcript.txt")

    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(transcript)
        logger.info(f"Transcript saved to {output_file}")
        return output_file
    except Exception as e:
        logger.error(f"Error saving transcript: {str(e)}")
        raise


In [14]:
def save_url_to_file(url):
    """
    Save the YouTube URL to a text file.

    Args:
        url (str): YouTube URL to save

    Returns:
        str: Path to the saved URL file
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"youtube_url_transcribed_{timestamp}.txt"
    url_file = os.path.join(full_path_nm, filename)

    try:
        with open(url_file, 'w') as f:
            f.write(f"YouTube URL: {url}\nProcessed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        logger.info(f"URL saved to: {url_file}")
        return url_file
    except Exception as e:
        logger.error(f"Error saving URL to file: {str(e)}")
        raise


In [15]:
def cleanup_files(files):
    """
    Clean up temporary files.

    Args:
        files (list): List of file paths to clean up
    """
    for file_path in files:
        try:
            if isinstance(file_path, str):
                file_path = Path(file_path)
            if file_path.exists():
                file_path.unlink()
                logger.info(f"Cleaned up: {file_path}")
        except Exception as e:
            logger.error(f"Error cleaning up {file_path}: {str(e)}")


In [16]:
def process_video(youtube_url=None, version_num=None, clean_dir=True, base_dir=None):
    """
    Process a YouTube video and create its transcript.

    Args:
        youtube_url (str, optional): YouTube URL to process
        version_num (str, optional): Version number for directory naming
        clean_dir (bool, optional): Whether to delete and recreate directory
        base_dir (str, optional): Base directory for storing files. If None, uses default.
    """
    global full_path_nm

    # Set base directory
    working_base_dir = base_dir if base_dir is not None else DEFAULT_BASE_DIR
    logger.info(f"Using base directory: {working_base_dir}")

    # Set version number
    version_number = version_num if version_num is not None else DEFAULT_VERSION

    # Setup directory
    full_path_nm = setup_directory(working_base_dir, version_number, clean_dir)

    # Change to working directory
    cwd = os.getcwd()
    if not os.path.samefile(cwd, full_path_nm):
        os.chdir(full_path_nm)
    logger.info(f"Working directory: {os.getcwd()}")

    # Default YouTube video URL if none provided
    if youtube_url is None or youtube_url.strip() == "":
        youtube_url = default_youtube_url
        logger.warning(f"No YouTube URL provided. Using default: {youtube_url}")

    audio_file = None
    chunk_paths = []
    created_files = []  # Track all created files
    permanent_files = []  # Track files that will be kept

    try:
        # Verify OpenAI API key
        if not OPENAI_API_KEY:
            raise ValueError("OpenAI API key not found in environment variables")

        # Generate timestamp for consistent file naming
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Download audio
        logger.info("Downloading audio...")
        audio_file = download_youtube_audio(youtube_url, output_path=full_path_nm)
        created_files.append(audio_file)
        logger.info(f"Audio downloaded: {audio_file}")

        # Split audio into chunks
        logger.info("Splitting audio into chunks...")
        chunk_paths = split_audio(Path(audio_file))
        created_files.extend([str(p) for p in chunk_paths])

        # Transcribe each chunk
        logger.info("Transcribing audio chunks...")
        full_transcript = []

        for i, chunk_path in enumerate(chunk_paths, 1):
            logger.info(f"Transcribing chunk {i}/{len(chunk_paths)}...")
            chunk_transcript = transcribe_audio_chunk(chunk_path, client)
            full_transcript.append(chunk_transcript)

        # Save transcript
        final_transcript = "\n\n".join(full_transcript)
        transcript_file = save_transcript(final_transcript)
        permanent_files.append(transcript_file)

        # Save URL to file
        try:
            url_file = save_url_to_file(youtube_url)
            permanent_files.append(url_file)
        except Exception as e:
            logger.error(f"Failed to save URL file: {e}")

        # Print summary of all files created during this run
        logger.info("\nFiles created in this run:")
        logger.info("\nPermanent files (kept in directory):")
        for file_path in permanent_files:
            logger.info(f"- {os.path.abspath(file_path)}")

        logger.info("\nTemporary files (will be cleaned up):")
        temp_files = [f for f in created_files if f not in permanent_files]
        for file_path in temp_files:
            logger.info(f"- {os.path.abspath(file_path)}")

    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")
        raise

    finally:
        # Clean up temporary files but keep the transcript and URL file
        logger.info("\nCleaning up temporary files...")
        cleanup_files([f for f in created_files if f not in permanent_files])
        if chunk_paths:
            chunks_dir = Path(chunk_paths[0]).parent
            try:
                chunks_dir.rmdir()
                logger.info("Cleaned up chunks directory")
            except Exception as e:
                logger.error(f"Error cleaning up chunks directory: {str(e)}")

        # Final summary of permanent files
        logger.info("\nFinal files stored in directory:")
        for file_path in permanent_files:
            logger.info(f"- {os.path.abspath(file_path)}")



In [17]:
if __name__ == "__main__":
    # Default execution - use all defaults
    process_video()

    # Example with custom base directory:
    # custom_base_dir = "/path/to/custom/directory"
    # process_video(base_dir=custom_base_dir)


[youtube] Extracting URL: https://www.youtube.com/watch?v=cdiD-9MMpb0
[youtube] cdiD-9MMpb0: Downloading webpage
[youtube] cdiD-9MMpb0: Downloading ios player API JSON
[youtube] cdiD-9MMpb0: Downloading mweb player API JSON
[youtube] cdiD-9MMpb0: Downloading player f8f53e1a
[youtube] cdiD-9MMpb0: Downloading m3u8 information
[info] cdiD-9MMpb0: Downloading 1 format(s): 251
[download] Destination: /content/drive/My Drive/python-projects/kaggle_experiments/transcriber/transcriber_v36/audio_20241214_221338.webm
[download] 100% of  167.01MiB in 00:00:03 at 44.85MiB/s  
[ExtractAudio] Destination: /content/drive/My Drive/python-projects/kaggle_experiments/transcriber/transcriber_v36/audio_20241214_221338.mp3
Deleting original file /content/drive/My Drive/python-projects/kaggle_experiments/transcriber/transcriber_v36/audio_20241214_221338.webm (pass -k to keep)
